# BERT Fine-Tuning with LoRA on SageMaker Local Environment

#### 환경설정 패키지 셋업

In [2]:
!pip install -q datasets transformers peft evaluate

In [1]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd

# 한국어 감성 분류 데이터 전처리

### 감성분석 데이터 불러오기 
- 네이버 영화 감성분석 데이터를 github에서 다운하고 bert peft lora 학습데이터로 사용합니다
- Train data : 150000 
- Test data : 50000
- All reviews are shorter than 140 characters
- Each sentiment class is sampled equally (i.e., random guess yields 50% accuracy)
-- 100K negative reviews (originally reviews of ratings 1-4)
-- 100K positive reviews (originally reviews of ratings 9-10)
-- Neutral reviews (originally reviews of ratings 5-8) are excluded

In [2]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1
Receiving objects: 100% (14763/14763), 56.19 MiB | 23.82 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [3]:
train = pd.read_table("nsmc/ratings_train.txt", converters={"document": str})
test = pd.read_table("nsmc/"+"ratings_test.txt", converters={"document": str})

In [4]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [5]:
# Train DataFrame to Dataset
train_dataset = Dataset.from_pandas(train)

# Test DataFrame to Dataset
test_dataset = Dataset.from_pandas(test)

In [6]:
train_dataset

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [7]:
train_dataset['document'][0:10]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.',
 '원작의 긴장감을 제대로 살려내지못했다.',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?']

### 훈련 데이터 셋의 긍 부정 비율 확인

In [8]:
# 훈련 데이터셋(train)에서 레이블(label)이 1인 데이터의 비율을 계산
np.array(train_dataset['label']).sum()/len(train_dataset['label'])

0.49884666666666666

# Huggingface Bert모델 로드 

- 한국어 선학습이 된 모델을 사용하시려면 아래 모델 사용을 추천드립니다.
    - 한국어 파인튜닝 전후의 확실한 변화를 확인하기 위해 기본 모델을 사용했습니다.
    - https://huggingface.co/klue/roberta-large
    - https://github.com/KLUE-benchmark/KLUE
- 추후, 기본 모델로 확인해보세요. : Facebook roberta-base
    - 한국어가 사전학습되지 않은 기본 facebook roberta의 경우 학습 수렴이 되지 않는 것을 확인 할 수 있습니다. 
    - https://huggingface.co/FacebookAI/roberta-base


In [9]:
# define label map
#레이블과 ID 간의 매핑을 정의 `id2label`은 ID(0, 1)에 대응하는 레이블 문자열("Negative", "Positive")을 매핑
#`label2id`는 반대로 레이블 문자열에 대응하는 ID를 매핑-> 이 매핑은 모델의 출력을 해석할 때 사용
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

In [10]:
#전 학습된 모델의 체크포인트에 사용한 모델은 RoBERTa 모델의 기본 버전입니다.
#model_checkpoint = 'roberta-base'klue/roberta-large
model_checkpoint = 'klue/roberta-large'


# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:

# display architecture
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

# 학습을 위한 data tokenizing 작업

### 입력 토큰 처리 
입력 시퀀스의 길이를 동일하게 만들어 배치 처리하기 위해 PAD 토큰을 추가합니다.

1. **가변 길이 시퀀스 처리**
트랜스포머 기반 모델은 고정된 길이의 입력을 요구합니다. 그러나 실제 데이터에서 문장의 길이는 가변적입니다. 패딩을 통해 모든 시퀀스의 길이를 동일하게 만들어 배치 처리가 가능해집니다.

2. **배치 처리 효율성**
딥러닝 모델에서 배치 처리는 연산 효율성을 높이는 중요한 기술입니다. 하지만 가변 길이 시퀀스는 배치 처리를 어렵게 만듭니다. 패딩을 통해 모든 시퀀스의 길이를 맞추면 텐서 연산을 효율적으로 수행할 수 있습니다.

3. **마스킹(Masking) 기법 활용**
패딩된 위치는 실제 토큰이 아니므로 모델이 학습하거나 예측하지 않도록 해야 합니다. 이를 위해 어텐션 마스크나 손실 마스크 등의 기법을 사용하는데, 패딩 토큰을 이용해 쉽게 마스킹할 수 있습니다.

4. **토크나이저 간소화**
일부 토크나이저는 패딩 토큰을 별도로 정의하지 않습니다. 이 경우 코드에서 명시적으로 패딩 토큰을 추가하여 토크나이저를 간소화할 수 있습니다.


example: 
- 배치 크기 : 2
- 최대 길이(maximun length) : 5

입력 문장:
1. "The quick brown fox"
2. "This is a long sentence"

토크나이저에 의해 토큰화된 결과:
1. ["The", "quick", "brown", "fox"]
- 문장의 길이 : 4
2. ["This", "is", "a", "long", "sentence"] 
- 문장의 길이 : 5


`[PAD]` 토큰 적용 후 결과:
1. ["The", "quick", "brown", "fox", "[PAD]"]
2. ["This", "is", "a", "long", "sentence"]


In [12]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [13]:

# create tokenize function
def tokenize_function(examples):
    # extract text
    #print("JHS TEST1 : ", examples)
    text = examples["document"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [14]:
train_dataset

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

## 학습문장 tokenizing

In [15]:
# tokenize training and validation datasets
tokenized_dataset = train_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

## 학습 데이터와 검증 데이터 나누기
- 학습 데이터 : train_tokenized_dataset (70%)
- 검증 데이터 : valid_tokenized_dataset (30%)

In [16]:
trainable_ratio = 0.7
# train/validation split
train_size = int(trainable_ratio * len(tokenized_dataset))
valid_size = len(tokenized_dataset) - train_size

train_tokenized_dataset = tokenized_dataset.select(range(train_size))
valid_tokenized_dataset = tokenized_dataset.select(range(train_size, train_size + valid_size))

print(f'Train dataset size: {len(train_tokenized_dataset)}')
print(f'Validation dataset size: {len(valid_tokenized_dataset)}')

Train dataset size: 105000
Validation dataset size: 45000


In [17]:
train_tokenized_dataset

Dataset({
    features: ['id', 'document', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 105000
})

In [18]:
train_tokenized_dataset['attention_mask'][0:1]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

## DataCollatorWithPadding 처리 

Hugging Face Transformers 라이브러리에서 제공하는 `DataCollatorWithPadding` 클래스를 사용하여 데이터 콜렉터(data collator)를 초기화합니다.

데이터 콜렉터는 데이터셋에서 샘플을 배치(batch)로 가져올 때, 각 배치의 샘플들을 적절하게 패딩(padding)하고 텐서(tensor)로 변환하는 역할을 합니다.

`DataCollatorWithPadding`은 다음과 같은 작업을 수행합니다:

1. **패딩(Padding)**: 배치 내 각 샘플의 길이가 다른 경우, 최대 길이에 맞추어 짧은 샘플들에 패딩(일반적으로 0)을 추가합니다. 이를 통해 배치 내 모든 샘플이 동일한 길이를 가지게 됩니다.

2. **텐서 변환(Tensor Conversion)**: 패딩이 완료된 샘플들을 PyTorch 또는 TensorFlow 텐서로 변환합니다. 이렇게 하면 딥러닝 모델이 텐서 형태의 입력을 받을 수 있습니다.

3. **주의 마스크(Attention Mask) 생성**: 각 샘플에 대해 주의 마스크(attention mask)를 생성합니다. 주의 마스크는 패딩 토큰을 무시하고 실제 토큰에만 주의를 기울이도록 하는 데 사용됩니다.

In [20]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='klue/roberta-large', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='left', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, 

# Evaluation

Hugging Face의 evaluate 라이브러리를 사용하여 "accuracy" 메트릭을 로드합니다.

evaluate 라이브러리의 accuracy는 validation-accuracy입니다.

In [22]:
# Load evaluation metrics
from evaluate import load

accuracy = load("accuracy")



In [23]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

# 파인튜닝되지 않은 모델을 감정분류에 적용

text_list 변수에 확인하고 싶은 테스트 문장을 정의한 다음, 파인튜닝 전과 후의 결과를 비교해봅니다.
이 블럭의 전모델은 Fine-tuning하지 않은 상태에서 실행되므로 예측 성능이 좋지 않을 수 있습니다. 일반적으로 BERT 모델을 특정 태스크에 적용하기 위해서는 Fine-tuning이 필요합니다.

In [24]:
# define list of examples
text_list = ["정말 오늘은 기쁜날이에요. 아빠를 사랑해요", 
             "우리는 지루한 하루를 보내고 있다", 
             "정말 최고에요. 너무 이뻐요.", 
             "영화가 긴장감이 전혀 없네.",
             "깊은 인상을 받았습니다. 감사하며 존경합니다.", 
             "내가 읽은 책 중에서 가장 지루하고 재미없는 책이다.",
            "진짜 짜증나네요. 목소리"]

In [25]:


print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])
     

Untrained model predictions:
----------------------------
정말 오늘은 기쁜날이에요. 아빠를 사랑해요 - Positive
우리는 지루한 하루를 보내고 있다 - Positive
정말 최고에요. 너무 이뻐요. - Positive
영화가 긴장감이 전혀 없네. - Positive
깊은 인상을 받았습니다. 감사하며 존경합니다. - Positive
내가 읽은 책 중에서 가장 지루하고 재미없는 책이다. - Positive
진짜 짜증나네요. 목소리 - Positive


# Train model

### PEFT LoRA
- 8bit 양자화 
이 설정은 일반적으로 PEFT를 사용하여 대규모 언어 모델을 미세 조정할 때 사용됩니다. 
LORA는 대규모 언어 모델을 효율적으로 파인튜닝(fine-tuning)하는 방법 중 하나로, 모든 매개변수를 업데이트하는 대신 작은 랭크(rank)의 업데이트만 수행합니다. LoRA를 적용하면 GPU 메모리 및 계산 리소스를 절약할 수 있습니다.
예를 들어, 이 설정으로 BERT 모델을 파인튜닝하면 BERT의 전체 매개변수를 업데이트하는 대신 랭크가 4인 작은 매개변수 업데이트만 수행하게 됩니다. 이렇게 하면 모델 성능을 유지하면서도 메모리 및 계산 리소스를 절약할 수 있습니다.


huggingface LoRA DOC
- https://huggingface.co/docs/peft/task_guides/lora_based_methods

huggingface PEFT github
- https://github.com/huggingface/peft


In [26]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['query'])


### Huggingface LoraConfig PEFT 라이브러리를 사용하여 시퀀스 분류 작업에 대한 LORA 설정을 초기화

`LoraConfig` 클래스는 LORA 설정을 정의하는 데 사용:
LoRA는 가중치 업데이트 행렬을 두 개의 작은 행렬로 분해합니다. 이러한 low-rank 행렬의 크기는 rank (r)에 의해 결정됩니다. 

1. `task_type="SEQ_CLS"`: 수행할 작업의 유형을 나타냅니다. `SEQ_CLS`는 시퀀스 분류(sequence classification) 작업을 의미합니다.
2. `r=4`: LORA 랭크(rank)를 설정합니다. 높은 랭크는 더 많은 메모리를 사용하지만 모델 성능을 향상시킬 수 있습니다.높을수록 모델에 훈련할 매개변수가 더 많다는 의미일 뿐만 아니라 모델의 학습 용량도 더 크다는 의미입니다. 
3. `lora_alpha=32`: LORA 스케일링 계수(scaling factor)를 설정합니다. 이 값은 LORA 매개변수의 크기를 조절합니다.
4. `lora_dropout=0.01`: LORA 드롭아웃(dropout) 비율을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하는 데 사용됩니다.
5. `target_modules=['query']`: LORA를 적용할 모듈을 지정합니다. 여기서는 'query' 모듈에 LORA를 적용하도록 설정되어 있습니다. 분해된 작은 행렬이 삽입되는 결정된 위치입니다.



'query' 모듈은 BERT의 Self-Attention 메커니즘에서 각 토큰에 대한 쿼리 벡터를 생성하는 역할을 합니다. LORA를 'query' 모듈에 적용하면 Self-Attention의 핵심 부분을 효율적으로 미세 조정할 수 있습니다.


BERT 모델에서 'query' 모듈은 Self-Attention 메커니즘의 일부로, 입력 시퀀스의 각 단어(토큰)에 대한 쿼리(query) 벡터를 생성하는 역할을 합니다.

BERT의 Self-Attention 메커니즘은 입력 시퀀스의 각 토큰과 다른 모든 토큰 간의 관계를 학습합니다. 이를 위해 각 토큰에 대해 쿼리(query), 키(key), 값(value) 벡터를 계산합니다.

1. **쿼리(Query) 벡터**: 현재 토큰에 대한 표현으로, 다른 토큰과의 관련성을 측정하는 데 사용됩니다.
2. **키(Key) 벡터**: 다른 토큰의 표현으로, 현재 토큰과의 관련성을 측정하는 데 사용됩니다.
3. **값(Value) 벡터**: 다른 토큰의 실제 정보(의미)를 포함하고 있습니다.

Self-Attention 메커니즘은 현재 토큰의 쿼리 벡터와 다른 토큰의 키 벡터 간의 유사도를 계산합니다. 이 유사도 점수를 사용하여 각 토큰의 값 벡터를 가중합하고, 가중합한 값 벡터들을 결합하여 현재 토큰의 새로운 표현을 만듭니다.

따라서 'query' 모듈은 입력 시퀀스의 각 토큰에 대한 쿼리 벡터를 생성하는 역할을 합니다. 이 쿼리 벡터는 Self-Attention 메커니즘에서 다른 토큰과의 관련성을 계산하는 데 사용됩니다.

`target_modules=['query']`를 설정하면, LORA는 'query' 모듈에 대해서만 작은 랭크의 업데이트를 수행합니다. 이렇게 함으로써 Self-Attention 메커니즘의 핵심인 쿼리 벡터를 효율적으로 미세 조정할 수 있습니다.

다른 모듈들(예: 키(key), 값(value) 모듈)은 원래 모델의 매개변수를 그대로 사용하게 됩니다. 이를 통해 전체 모델의 대부분의 매개변수를 동결한 채 Self-Attention 메커니즘의 일부만 효율적으로 파인튜닝할 수 있습니다.

In [27]:
peft_config


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'query'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

### PEFT를 적용할 파인튜닝할 모델에서 실제로 학습 가능한 파라미터들을 조회

In [28]:

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,248,258 || all params: 337,906,692 || trainable%: 0.3694090793561437


### 하이퍼파라미터 설정

In [29]:
# hyperparameters
lr = 1e-3
batch_size = 16
num_epochs = 3

In [30]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

#### 콜백함수를 확인해보면 evaluate 라이브러리의 accuracy는 eval_dataset의 validation-accuracy를 출력하는것을 확인할수 있습니다.

In [31]:
from transformers import TrainerCallback

class MyCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'eval_accuracy' in logs:
            print("? : ", logs)
            print(f"Epoch {state.epoch} - Validation Accuracy: {logs['eval_accuracy']}")

In [32]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=valid_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[MyCallback]
)

# train model
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.417000,0.392200,{'accuracy': 0.8377111111111111}
2,0.385800,0.346977,{'accuracy': 0.8587777777777778}
3,0.340900,0.303850,{'accuracy': 0.8744}


? :  {'eval_loss': 0.3922002613544464, 'eval_accuracy': {'accuracy': 0.8377111111111111}, 'eval_runtime': 171.7605, 'eval_samples_per_second': 261.993, 'eval_steps_per_second': 16.377, 'epoch': 1.0}
Epoch 1.0 - Validation Accuracy: {'accuracy': 0.8377111111111111}
? :  {'eval_loss': 0.3469771146774292, 'eval_accuracy': {'accuracy': 0.8587777777777778}, 'eval_runtime': 171.836, 'eval_samples_per_second': 261.878, 'eval_steps_per_second': 16.37, 'epoch': 2.0}
Epoch 2.0 - Validation Accuracy: {'accuracy': 0.8587777777777778}
? :  {'eval_loss': 0.3038501441478729, 'eval_accuracy': {'accuracy': 0.8744}, 'eval_runtime': 171.8319, 'eval_samples_per_second': 261.884, 'eval_steps_per_second': 16.371, 'epoch': 3.0}
Epoch 3.0 - Validation Accuracy: {'accuracy': 0.8744}


TrainOutput(global_step=19689, training_loss=0.38777106976435227, metrics={'train_runtime': 3193.6178, 'train_samples_per_second': 98.634, 'train_steps_per_second': 6.165, 'total_flos': 3.620195045984218e+16, 'train_loss': 0.38777106976435227, 'epoch': 3.0})

## 한국어 데이터 LoRA 파인튜닝 후 결과 확인
- 파인튜닝된 Bert 모델의 감정(2진분류)가 잘 되는 것을 확인 할 수 있습니다.

In [33]:
model.to('cpu')

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
정말 오늘은 기쁜날이에요. 아빠를 사랑해요 - Positive
우리는 지루한 하루를 보내고 있다 - Negative
정말 최고에요. 너무 이뻐요. - Positive
영화가 긴장감이 전혀 없네. - Negative
깊은 인상을 받았습니다. 감사하며 존경합니다. - Positive
내가 읽은 책 중에서 가장 지루하고 재미없는 책이다. - Negative
진짜 짜증나네요. 목소리 - Negative
